In [41]:
import spacy
from util.RestCaller import callAllenNlpApi

nlp = spacy.blank("en")
#nlp.add_pipe('opentapioca')
doc = nlp("Japan began the defence of their title with a lucky 2-1 win against Syria in a championship match on Friday.")
#for span in doc.ents:
 #   print((span.text, span.kb_id_, span.label_, span._.description, span._.score))

#print(doc.text)

result = callAllenNlpApi("coreference-resolution", "the quick brown fox jumps over the lazy dog." )

print(result)

<!doctype html>
<html lang=en>
  <head>
    <title>KeyError: 'document'
 // Werkzeug Debugger</title>
    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css">
    <link rel="shortcut icon"
        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">
    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>
    <script>
      var CONSOLE_MODE = false,
          EVALEX = true,
          EVALEX_TRUSTED = false,
          SECRET = "8Dj7WqcH86kJnqhNJx2T";
    </script>
  </head>
  <body style="background-color: #fff">
    <div class="debugger">
<h1>KeyError</h1>
<div class="detail">
  <p class="errormsg">KeyError: &#39;document&#39;
</p>
</div>
<h2 class="traceback">Traceback <em>(most recent call last)</em></h2>
<div class="traceback">
  <h3></h3>
  <ul><li><div class="frame" id="frame-139941515322752">
  <h4>File <cite class="filename">"/usr/local/lib/python3.8/site-packages/flask/app.py"</cite>,
      line <em class="line">2095<

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [5]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_trf")
text = """The UK's FTSE-100 index fell 232.90 points to 6038.30, and Japan's Nikkei 225 fell 406.51 points to 16764.09."""
#text = """in a mass sell-off stemming from the sub-prime mortgage crisis in the United States"""
doc = nlp(text)
sentence_spans = list(doc.sents)
displacy.render(doc, style="dep")

In [8]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_trf")
text = """In ancient Rome, some neighbors live in three adjacent houses. In the center is the house of Senex, who lives there with wife Domina, son Hero, and several slaves, including head slave Hysterium and the musical's main character Pseudolus. A slave belonging to Hero, Pseudolus wishes to buy, win, or steal his freedom. One of the neighboring houses is owned by Marcus Lycus, who is a buyer and seller of beautiful women; the other belongs to the ancient Erronius, who is abroad searching for his long-lost children (stolen in infancy by pirates). One day, Senex and Domina go on a trip and leave Pseudolus in charge of Hero. Hero confides in Pseudolus that he is in love with the lovely Philia, one of the courtesans in the House of Lycus (albeit still a virgin)."""
doc = nlp(text)
sentence_spans = list(doc.sents)
displacy.serve(sentence_spans, style="dep", port=5459)

/home/neo/environments/text2graphs/venv/lib/python3.8/site-packages/spacy/displacy/__init__.py:108: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5459 ...

Shutting down server on port 5459.


In [16]:
import spacy
from util.CallAllenNlpCoref import callAllenNlpCoref

nlp = spacy.blank("en")
#nlp.add_pipe('opentapioca')
#doc = nlp("john bought a new fast 4-wheel car. The car became the slow one in a year and he is sad.  He will buy a new car in next year.  ")
doc = nlp("john bought a new fast 4-wheel car.")
#for span in doc.ents:
 #   print((span.text, span.kb_id_, span.label_, span._.description, span._.score))

#print(doc.text)

result = callAllenNlpCoref("coreference-resolution", doc.text )

print("clusters: " , result["clusters"])

for cluster in result["clusters"]:
    #print("cluster: ", cluster)
    i = 0
    reference = ""
    for span in cluster:
        if i == 0:
            i+=1
            # the first span will be the reference point for all other coreferences link
            reference = span[0]
            continue
        mention = {'from_index': span[0], 'to_index': reference}
        print (mention)
        #print (chunk)

    #mention = {'from_index': cluster.mentions[-1].start_char, 'to_index': cluster.mentions[0].start_char}
    




#print(result)

clusters:  []


In [8]:
import spacy
from py2neo import Graph
from py2neo import *
from util.CallAllenNlpCoref import callAllenNlpCoref
from spacy.tokens import Doc, Token, Span



def store_coref_mentions(doc, mentions):
    uri = 'bolt://10.1.54.74:7687'
    username = 'neo4j'
    password = 'neo123'
    graph = Graph(uri, auth=(username, password))

    # create the referrant span , attaches it with the tagOccurrences
    # identify the namedEntity that belongs to the antecedent
    # 

    sg=""
    PARTICIPANT = Relationship.type("PARTICIPANT")
    PARTICIPATES_IN = Relationship.type("PARTICIPATES_IN")
    MENTIONS = Relationship.type("MENTIONS")

    for mention in mentions:
        
        start_index = mention['referent']['start_index']
        end_index = mention['referent']['end_index']
        start_index_antecedent = mention['antecedent']['end_index']
        end_index_antecedent = mention['antecedent']['end_index']

        # create a corefMention node
        corefMention_node = Node("CorefMention", text=mention['referent']['text'], startIndex=start_index, endIndex=end_index)
        
        # connect the corefMention node with all the participating tagOccurrences
        index_range = range(start_index, end_index)
        for index in index_range:
            query = "match (x:TagOccurrence {tok_index_doc:" + str(index) + "})-[:HAS_TOKEN]-()-[:CONTAINS_SENTENCE]-(:AnnotatedText {id:"+str(doc._.text_id)+"}) return x"
            token_node= graph.evaluate(query)
            token_mention_rel = PARTICIPATES_IN(token_node,corefMention_node)
            if sg == "":
                sg = token_mention_rel
            else:
                sg = sg | token_mention_rel
        
        # connect the corefMention node with the antecdent namedEntity. 
        np_query = """MATCH (document:AnnotatedText {id:"""+str(doc._.text_id)+"""})-[*2]->(np:TagOccurrence)-[:PARTICIPATES_IN]->(end:NamedEntity) 
            WHERE np.index = """ +start_index_antecedent + """
            RETURN np"""
        np_node = graph.evaluate(np_query)
        if np_node is None:
            try:
                graph.create(sg)
            except BaseException as err:
                print(f"Unexpected {err=}, {type(err)=}")
            
            continue
        
        coref_mention_np_rel = MENTIONS(corefMention_node,np_node)
        
        sg = sg | coref_mention_np_rel
        
        try:
            graph.create(sg)
        except BaseException as err:
            print(f"Unexpected {err=}, {type(err)=}") 
    return sg

if not Doc.has_extension("text_id"):
    Doc.set_extension("text_id", default=1)


nlp = spacy.blank("en")
nlp = spacy.load('en_core_web_trf')
#nlp.add_pipe('opentapioca')
#doc = nlp("john bought a new fast 4-wheel car. The car became the slow one in a year and he is sad.  He will buy a new car in next year.  ")
doc = nlp("john bought a new fast 4-wheel car. He drives the car very fast.")
#for span in doc.ents:
 #   print((span.text, span.kb_id_, span.label_, span._.description, span._.score))

#print(doc.text)

result = callAllenNlpCoref("coreference-resolution", doc.text )

#print("clusters: " , result["clusters"])
# storing the coreference mentions as graph nodes linked with antecedent via mentions edges
        # steps
        # 1. get the coref-mention and antedent pair
coref = []
for cluster in result["clusters"]:
    i=0
    antecedent_span = ""
    for span_token_indexes in cluster:
        if i == 0:
            i+=1
            # the first span will be the antecedent for all other references
            antecedent_span = doc[span_token_indexes[0]:span_token_indexes[-1]+1]
            antecedent_node = {'start_index': span_token_indexes[0], 'end_index': span_token_indexes[-1]+1, 'text': antecedent_span}
            continue
        coref_mention_span = doc[span_token_indexes[0]:span_token_indexes[-1]+1]
        coref_mention_node = {'start_index': span_token_indexes[0], 'end_index': span_token_indexes[-1]+1, 'text': coref_mention_span}
        #mention = {'from_index': span[-1], 'to_index': antecedent}
        #mention = { 'referent': coref_mention_span, 'antecedent': antecedent_span}
        mention = { 'referent': coref_mention_node, 'antecedent': antecedent_node}
        coref.append(mention)
    
    print(coref)

    print(store_coref_mentions(doc, coref))

/home/neo/environments/text2graphs/venv/lib/python3.8/site-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_trf' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/home/neo/environments/text2graphs/venv/lib/python3.8/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[{'referent': {'start_index': 10, 'end_index': 11, 'text': He}, 'antecedent': {'start_index': 0, 'end_index': 1, 'text': john}}]


AttributeError: 'NoneType' object has no attribute 'labels'

In [20]:
import spacy

nlp = spacy.blank("en")
#nlp.add_pipe('opentapioca')
#doc = nlp("john bought a new fast 4-wheel car. The car became the slow one in a year and he is sad.  He will buy a new car in next year.  ")
nlp = spacy.load('en_core_web_trf')
doc = nlp("""Jim Cramer, of CNBC's Mad Money, remarked that as many as seven million people will lose their homes from bad mortgages. Last Friday, Cramer went on a tirade on CNBC's Street Signs, saying that the "Fed was asleep" and called for them to lower rates immediately.""")

for chunk in doc.noun_chunks:
    print("chunk: ", chunk.text, "root: ", chunk.root.text, "root dep: ", chunk.root.dep_,
            "head: ", chunk.root.head.text)

/home/neo/environments/text2graphs/venv/lib/python3.8/site-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_trf' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


chunk:  Jim Cramer root:  Cramer root dep:  nsubj head:  remarked
chunk:  CNBC's Mad Money root:  Money root dep:  pobj head:  of
chunk:  as many as seven million people root:  people root dep:  nsubj head:  lose
chunk:  their homes root:  homes root dep:  dobj head:  lose
chunk:  bad mortgages root:  mortgages root dep:  pobj head:  from
chunk:  Cramer root:  Cramer root dep:  nsubj head:  went
chunk:  a tirade root:  tirade root dep:  pobj head:  on
chunk:  CNBC's Street Signs root:  Signs root dep:  pobj head:  on
chunk:  the "Fed root:  Fed root dep:  nsubj head:  was
chunk:  them root:  them root dep:  nsubj head:  lower
chunk:  rates root:  rates root dep:  dobj head:  lower


/home/neo/environments/text2graphs/venv/lib/python3.8/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [3]:
import xml.etree.ElementTree as ET
tree = ET.parse('/home/neo/environments/text2graphs/text2graphs/dataset/76437_Markets_dragged_down_by_credit_crisis.naf')
root = tree.getroot()

In [4]:
root.tag

'NAF'

In [17]:
root[1].text

'\n\nGlobal stock markets fell today, in a mass sell-off stemming from the sub-prime mortgage crisis in the United States. The Dow Jones Industrial Average rebounded late in the day after falling more than 250 points, ending the day down about 31 points. The UK\'s FTSE-100 index fell 232.90 points to 6038.30, and Japan\'s Nikkei 225 fell 406.51 points to 16764.09. \n\nCentral banks across the world are injecting funds into their banking systems to add liquidity, fearing that many financial firms with subprime ties will be insolvent. Yesterday, the U.S. Federal Reserve transferred US$24 billion to temporary reserves, following the European Central Bank, which authorized a record €83.6 billion addition to its banks, its biggest cash infusion ever. On Friday, the Fed entered into a $38 billion repurchase agreement of mortgage-backed securities, easing stockholder worries. Also on Friday, the Bank of Japan injected ¥1 trillion into Japan\'s financial system. \n\nThe Federal Reserve met thi

In [3]:
import spacy
nlp=spacy.load("en_core_web_trf")
doc=nlp("This is a sentence written in english by a native English speaker.")


for tok in doc:
    print( tok.idx, len(tok.text), len(tok.text)+tok.idx)

/home/neo/environments/text2graphs/venv/lib/python3.8/site-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_trf' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


0 4 4
5 2 7
8 1 9
10 8 18
19 7 26
27 2 29
30 7 37
38 2 40
41 1 42
43 6 49
50 7 57
58 7 65
65 1 66


/home/neo/environments/text2graphs/venv/lib/python3.8/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [1]:
text = """



The biggest U.S. stock market index, the Dow Jones, plunged by more than 416 points by the closing bell on Tuesday, the worst single-day decline since the re-opening of the markets following the September 11th terrorist attacks. The slip followed a 9% sell-off in Chinese markets. Other stock indices, such as the Nasdaq and the S&amp;P 500, dipped by 96 points and 50 points, respectively. It was the 7th largest single-day decline is U.S. history.

Analysts blamed the slide on a number of factors: (1) U.S. and Chinese stock exchanges had powered upward for too long and were due for a significant correction; (2) inflation and the prices of bonds and equities had become too high; (3) the U.S. is headed for an economic recession. Alan Greenspan indicated on Monday that the U.S. economy would not sustain its recent growth, where the Dow Industrials set day-to-day records and the White House signaled on Tuesday that it would revise its estimate of fourth-quarter 2006 GDP growth down to an annual rate of about 2.3 percent from an initial forecast of 3.5 percent and was increasingly nervous that the figure could come in even lower.
Around 3 p.m. EST, Reuters reported that the New York Stock Exchange had implemented on-floor tradition restrictions meant to curb the sell off.

The decline also occurred on the day that a plot to assassinate U.S. Vice President Richard Cheney failed in Afghanistan.

"""

#text = text.replace('\n\n','. ')

text = text.replace('\n','')

print(text)
print("")


import spacy
nlp=spacy.load("en_core_web_trf")
doc=nlp(text)

i=0


for sent in doc.sents:
    print(sent.start," : ", sent.text)



. . The biggest U.S. stock market index, the Dow Jones, plunged by more than 416 points by the closing bell on Tuesday, the worst single-day decline since the re-opening of the markets following the September 11th terrorist attacks. The slip followed a 9% sell-off in Chinese markets. Other stock indices, such as the Nasdaq and the S&amp;P 500, dipped by 96 points and 50 points, respectively. It was the 7th largest single-day decline is U.S. history.. Analysts blamed the slide on a number of factors: (1) U.S. and Chinese stock exchanges had powered upward for too long and were due for a significant correction; (2) inflation and the prices of bonds and equities had become too high; (3) the U.S. is headed for an economic recession. Alan Greenspan indicated on Monday that the U.S. economy would not sustain its recent growth, where the Dow Industrials set day-to-day records and the White House signaled on Tuesday that it would revise its estimate of fourth-quarter 2006 GDP growth down to an

/home/neo/environments/text2graphs/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/neo/environments/text2graphs/venv/lib/python3.8/site-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_trf' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/home/neo/environments/text2graphs/venv/lib/python3.8/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not av

0  :  .
1  :  .
2  :  The biggest U.S. stock market index, the Dow Jones, plunged by more than 416 points by the closing bell on Tuesday, the worst single-day decline since the re-opening of the markets following the September 11th terrorist attacks.
47  :  The slip followed a 9% sell-off in Chinese markets.
60  :  Other stock indices, such as the Nasdaq and the S&amp;P 500, dipped by 96 points and 50 points, respectively.
83  :  It was the 7th largest single-day decline is U.S. history..
96  :  Analysts blamed the slide on a number of factors: (1) U.S. and Chinese stock exchanges had powered upward for too long and were due for a significant correction; (2) inflation and the prices of bonds and equities had become too high; (3) the U.S. is headed for an economic recession.
156  :  Alan Greenspan indicated on Monday that the U.S. economy would not sustain its recent growth, where the Dow Industrials set day-to-day records and the White House signaled on Tuesday that it would revise its

In [11]:
text = """. The biggest U.S. stock market index, the Dow Jones, plunged by more than 416 points by the closing bell on Tuesday, the worst single-day decline since the re-opening of the markets following the September 11th terrorist attacks. The slip followed a 9% sell-off in Chinese markets. Other stock indices, such as the Nasdaq and the S&amp;P 500, dipped by 96 points and 50 points, respectively. It was the 7th largest single-day decline is U.S. history.

Analysts blamed the slide on a number of factors: (1) U.S. and Chinese stock exchanges had powered upward for too long and were due for a significant correction; (2) inflation and the prices of bonds and equities had become too high; (3) the U.S. is headed for an economic recession. Alan Greenspan indicated on Monday that the U.S. economy would not sustain its recent growth, where the Dow Industrials set day-to-day records and the White House signaled on Tuesday that it would revise its estimate of fourth-quarter 2006 GDP growth down to an annual rate of about 2.3 percent from an initial forecast of 3.5 percent and was increasingly nervous that the figure could come in even lower.

Around 3 p.m. EST, Reuters reported that the New York Stock Exchange had implemented on-floor tradition restrictions meant to curb the sell off.

The decline also occurred on the day that a plot to assassinate U.S. Vice President Richard Cheney failed in Afghanistan.

"""

import spacy
nlp=spacy.load("en_core_web_trf")
doc=nlp(text)

for tok in doc:
    print(tok.idx," : ", tok.text)

/home/neo/environments/text2graphs/venv/lib/python3.8/site-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_trf' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/home/neo/environments/text2graphs/venv/lib/python3.8/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


0  :  .
2  :  The
6  :  biggest
14  :  U.S.
19  :  stock
25  :  market
32  :  index
37  :  ,
39  :  the
43  :  Dow
47  :  Jones
52  :  ,
54  :  plunged
62  :  by
65  :  more
70  :  than
75  :  416
79  :  points
86  :  by
89  :  the
93  :  closing
101  :  bell
106  :  on
109  :  Tuesday
116  :  ,
118  :  the
122  :  worst
128  :  single
134  :  -
135  :  day
139  :  decline
147  :  since
153  :  the
157  :  re
159  :  -
160  :  opening
168  :  of
171  :  the
175  :  markets
183  :  following
193  :  the
197  :  September
207  :  11th
212  :  terrorist
222  :  attacks
229  :  .
231  :  The
235  :  slip
240  :  followed
249  :  a
251  :  9
252  :  %
254  :  sell
258  :  -
259  :  off
263  :  in
266  :  Chinese
274  :  markets
281  :  .
283  :  Other
289  :  stock
295  :  indices
302  :  ,
304  :  such
309  :  as
312  :  the
316  :  Nasdaq
323  :  and
327  :  the
331  :  S&amp;P
339  :  500
342  :  ,
344  :  dipped
351  :  by
354  :  96
357  :  points
364  :  and
368  :  50
371  :  points


In [7]:
import spacy
from spacy.language import Language

@Language.component("custom_sentencizer")
def custom_sentencizer(doc):
    for tok in doc:
        # Define sentence start if pipe + titlecase token
        if doc[tok.i].is_title:
            doc[tok.i].is_sent_start = True
        else:
            # Explicitly set sentence start to False otherwise, to tell
            # the parser to leave those tokens alone
            doc[tok.i].is_sent_start = False
        # if tok.is_space:
        #     doc[tok.i].is_sent_start = False
    return doc

text = """

The biggest U.S. stock market index, the Dow Jones, plunged by more than 416 points by the closing bell on Tuesday, the worst single-day decline since the re-opening of the markets following the September 11th terrorist attacks. The slip followed a 9% sell-off in Chinese markets. Other stock indices, such as the Nasdaq and the S&amp;P 500, dipped by 96 points and 50 points, respectively. It was the 7th largest single-day decline is U.S. history.

Analysts blamed the slide on a number of factors: (1) U.S. and Chinese stock exchanges had powered upward for too long and were due for a significant correction; (2) inflation and the prices of bonds and equities had become too high; (3) the U.S. is headed for an economic recession. Alan Greenspan indicated on Monday that the U.S. economy would not sustain its recent growth, where the Dow Industrials set day-to-day records and the White House signaled on Tuesday that it would revise its estimate of fourth-quarter 2006 GDP growth down to an annual rate of about 2.3 percent from an initial forecast of 3.5 percent and was increasingly nervous that the figure could come in even lower.

Around 3 p.m. EST, Reuters reported that the New York Stock Exchange had implemented on-floor tradition restrictions meant to curb the sell off.

The decline also occurred on the day that a plot to assassinate U.S. Vice President Richard Cheney failed in Afghanistan.

"""
nlp = spacy.load("en_core_web_trf")
nlp.add_pipe("custom_sentencizer", before="parser")  # Insert before the parser
doc = nlp(text)
for sent in doc.sents:
    print(sent.start, ": ", sent.text)

/home/neo/environments/text2graphs/venv/lib/python3.8/site-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_trf' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/home/neo/environments/text2graphs/venv/lib/python3.8/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


0 :  


1 :  The biggest
3 :  U.S. stock market index, the
9 :  Dow
10 :  Jones, plunged by more than 416 points by the closing bell on
23 :  Tuesday, the worst single-day decline since the re-opening of the markets following the
41 :  September 11th terrorist attacks.
46 :  The slip followed a 9% sell-off in
56 :  Chinese markets.
59 :  Other stock indices, such as the
66 :  Nasdaq and the S&amp;P 500, dipped by 96 points and 50 points, respectively.
82 :  It was the 7th largest single-day decline is
92 :  U.S. history.


96 :  Analysts blamed the slide on a number of factors: (1)
109 :  U.S. and
111 :  Chinese stock exchanges had powered upward for too long and were due for a significant correction; (2) inflation and the prices of bonds and equities had become too high; (3) the
148 :  U.S. is headed for an economic recession.
156 :  Alan
157 :  Greenspan indicated on
160 :  Monday that the
163 :  U.S. economy would not sustain its recent growth, where the
174 :  Dow
175 :  Industrial

In [3]:
f = open("../text2graphs/tarsqi-dataset/myfile.txt", "x")
f.write("this is demo file")
f.close()

In [6]:
value = "2007-08-10T00:00:00"
value = value[0:10]
value= value.replace('-','')
print(value)



20070810


In [8]:
import spacy 
nlp = spacy.load("en_core_web_trf")
text = """Global stock markets fell today, in a mass sell-off stemming from the sub-prime mortgage crisis in the United States. The Dow Jones Industrial Average rebounded late in the day after falling more than 250 points, ending the day down about 31 points. The UK's FTSE-100 index fell 232.90 points to 6038.30, and Japan's Nikkei 225 fell 406.51 points to 16764.09. 

Central banks across the world are injecting funds into their banking systems to add liquidity, fearing that many financial firms with subprime ties will be insolvent. Yesterday, the U.S. Federal Reserve transferred US$24 billion to temporary reserves, following the European Central Bank, which authorized a record €83.6 billion addition to its banks, its biggest cash infusion ever. On Friday, the Fed entered into a $38 billion repurchase agreement of mortgage-backed securities, easing stockholder worries. Also on Friday, the Bank of Japan injected ¥1 trillion into Japan's financial system. 

The Federal Reserve met this week, but decided to maintain its target rate of 5.25%, although on Friday the federal funds rate was hovering around 6%, indicating a drop in liquidity. 

The volatile week began last Friday with Bear Stearns tumbling as a result from its complete loss of two major hedge funds worth more than $1.5 billion. The hedge funds had been dangerously exposed to the massive sub-prime mortgage failure, and the company announced it was unable to return any money to investors. 

Washington Mutual, and Countrywide Financial, both very large U.S. home loan lenders, saw shares fall. Countrywide Financial made a statement this week, saying they will be forced to retain a greater proportion of mortgage. American Home Mortgage Investment Corp, another large lender, recently filed for bankruptcy. The U.S. housing market has been declining for more than two years after the Federal Reserve raised interest rates 17 times. Now, lenders are in a quagmire from millions of people who are unable to repay loans after taking adjustable rate mortgages, teaser rates, interest-only mortgages, or piggyback rates. 

Jim Cramer, of CNBC's Mad Money, remarked that as many as seven million people will lose their homes from bad mortgages. Last Friday, Cramer went on a tirade on CNBC's Street Signs, saying that the "Fed was asleep" and called for them to lower rates immediately. 

Asian and European markets have become increasingly entangled in the subprime mortgage crisis in the U.S. Deutsche Bank of Germany lost almost $3.5 billion in share value, forcing the government to organize a bail-out. France's largest bank, BNP Paribas SA, halted withdrawals from three large investment funds which were crippled by sub-prime exposure.
"""

text2 = """Jim Cramer, of CNBC's Mad Money"""
doc = nlp(text2)

for chunk in doc.noun_chunks:
    print(chunk.text, "head: ", chunk.root.text)



/home/neo/environments/text2graphs/venv/lib/python3.8/site-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_trf' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Jim Cramer head:  Cramer
CNBC's Mad Money head:  Money


/home/neo/environments/text2graphs/venv/lib/python3.8/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [6]:
import spacy 

nlp = spacy.load("../corpus/output/model-best")
doc = nlp("Proofpoint report mentions that the German-language messages were turned off once the UK messages were established, indicating a conscious effort to spread FluBot 446833e3f8b04d4c3c2d2288e456328266524e396adbfeba3769d00727481e80 and DroidJack RAT in Android phones.")
#print(doc.ents)


for ent in doc.ents:
    print(ent.text, ent.label_)  

Proofpoint Organization
DroidJack RAT Malware
Android System
